<a href="https://colab.research.google.com/github/chihoang811/chihoang811/blob/main/PageRank_based_Link_analysis_on_Books/Book_ratings_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***PageRank based Link Analysis on Books*** - Linh Chi HOANG

*(Considering the similarity in book titles)*


---



##### **1. Download Datasets**


---




*   Please access your Kaggle account and download your API tokens for uploading the kaggle.json file and downloading the datasets
*   For this project, only rating dataset was used



In [ ]:
from google.colab import files
print("Please upload your kaggle.json file.")
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle

In [4]:
!kaggle datasets download mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:09<00:00, 287MB/s]
100% 1.06G/1.06G [00:09<00:00, 126MB/s]


In [5]:
!unzip amazon-books-reviews.zip

Archive:  amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


In [6]:
import pandas as pd
book_rating = pd.read_csv("Books_rating.csv")

##### **2. Parameters settings**


---


*   Note: Because of the limiation in RAM, this project used only popular books, which have at least 81 unique users




In [24]:
USE_SUBSAMPLE = True
MIN_RATING = 4 # keep onlybooks with score >= 4
MIN_USERS_PER_BOOK = 81 # choose only books rated by at least 81 users
MIN_SHARED_USERS_FOR_EDGES = 2 # edges are created only if for each pair of book there are at least 2 mutual users rating them
JACC_THRESHOLD = 0.7
LSH_THRESHOLD = 0.5
NUM_PERM =128 # number of permutation

##### **3. Data Preprocessing with dataset *book_ratings***


---


*   Choose only books with **review/score ≥ 4.0**
*   Check NAN values
*   Normalize book titles
*   Check similar book titles (using MinHash signatures (with LSH) and Jaccard similarity)
*   Create Weight matrix W
*   Create Transition matrix M

In [8]:
# Check for NaN values
nan_check = book_rating.isnull()
print(nan_check.sum())

# Remove rows with NaN values
book_rating = book_rating.dropna()

Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64


In [9]:
# Normalize the titles (Because there are some books whose titles are the same but written in different forms)
import re

def normalized_title(t):
    t = t.lower().strip()
    t = re.sub(r'[^a-z0-9 ]+', '', t)   # remove punctuation
    t = re.sub(r'\s+', ' ', t)          # collapse spaces
    return t

book_rating['Title_cleaned'] = book_rating['Title'].apply(normalized_title)

# Filter ratings (>= 4) and select Title
filtered_books = book_rating[book_rating['review/score'] >= MIN_RATING][['Title_cleaned', 'User_id', 'review/score']]

In [10]:
# Count how many unique users rated each book
book_user_counts = filtered_books.groupby("Title_cleaned")["User_id"].nunique()

if USE_SUBSAMPLE:
  # Keep only books with at least 81 unique users
  popular_books = book_user_counts[book_user_counts >= MIN_USERS_PER_BOOK].index
  filtered_books = filtered_books[filtered_books["Title_cleaned"].isin(popular_books)]
else:
 popular_books = book_user_counts.index

In [11]:
filtered_books

,Title_cleaned,User_id,review/score
38660,gods and kings chronicles of the kings 1,A7IMBNFYANPNV,5.0
38661,gods and kings chronicles of the kings 1,A1WV4Q44JE40UF,5.0
38662,gods and kings chronicles of the kings 1,A32MYDPSMCHT1L,5.0
38663,gods and kings chronicles of the kings 1,AAOJ5T6VS5Z6O,5.0
38664,gods and kings chronicles of the kings 1,A2X1OYBRURS04R,5.0
...,...,...,...
2994351,first 100 words bright baby,AIFSOGUNXFADJ,5.0
2994352,first 100 words bright baby,A2B9B5F9FQ4JJ8,4.0
2994354,first 100 words bright baby,A2G9EZ3R716RH1,4.0
2994355,first 100 words bright baby,A37P45ZC5DSAJJ,5.0


###### *Apllying MinHash (with LSH), and Jaacard similarity*

In [12]:
# Number of unique titles
nunique_titles = filtered_books["Title_cleaned"].nunique()
print(nunique_titles)

378


In [13]:
# Convert each normalized title into a set of words
def title_to_set(title):
    return set(title.split())

In [14]:
titles = filtered_books["Title_cleaned"].unique()
title_tokens = {t: title_to_set(t) for t in titles}

In [15]:
!pip install datasketch

In [25]:
# Create MinHash signatures
from datasketch import MinHash

def build_minhash(token_set, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for token in token_set:
        m.update(token.encode("utf8"))
    return m

minhashes = {t: build_minhash(tok, num_perm=NUM_PERM) for t, tok in title_tokens.items()}

In [26]:
# Build LSH
from datasketch import MinHashLSH

lsh = MinHashLSH(
    threshold = LSH_THRESHOLD,
    num_perm = NUM_PERM
)

for title, mh in minhashes.items():
    lsh.insert(title, mh)

In [27]:
# Retrieve similar titles
candidate_pairs = set()

for title, mh in minhashes.items():
    for candidate in lsh.query(mh):
        if title != candidate:  # to avoid duplicates
            candidate_pairs.add(tuple(sorted((title, candidate))))

print("Candidate pairs:", len(candidate_pairs))

Candidate pairs: 53


In [28]:
# Jaccard similarity function
def jaccard_similarity(set1, set2):
    union = set1 | set2
    if not union:
        return 1.0 #identical set
    return len(set1 & set2) / len(union)

In [29]:
# Apply Jaccard function
similarities = []

for t1, t2 in candidate_pairs:
    sim = jaccard_similarity(title_tokens[t1], title_tokens[t2])
    similarities.append((t1, t2, sim))

print("Verified similar pairs:", len(similarities))

similar_df = pd.DataFrame(similarities, columns=['Title1', 'Title2', 'Sim']).sort_values("Sim", ascending=False)
similar_df

Verified similar pairs: 53


,Title1,Title2,Sim
24,the picture of dorian gray classic collection ...,the picture of dorian gray the classic collection,0.777778
26,the picture of dorian gray,the picture of dorian gray the classic collection,0.714286
23,lord of chaos turtleback school library bindin...,shadow rising turtleback school library bindin...,0.692308
8,harrington on hold em expert strategy for no l...,harrington on hold em expert strategy for no l...,0.687500
4,a crown of swords turtleback school library bi...,lord of chaos turtleback school library bindin...,0.642857
43,a crown of swords turtleback school library bi...,shadow rising turtleback school library bindin...,0.642857
44,exodus turtleback school library binding edition,the lorax turtleback school library binding ed...,0.625000
34,exodus turtleback school library binding edition,maniac magee turtleback school library binding...,0.625000
35,the lorax turtleback school library binding ed...,the tao of pooh turtleback school library bind...,0.600000
39,up from slavery,up from slavery an autobiography,0.600000


###### *Merge titles whose Jaccard similarity ≥ 0.4*

In [30]:
# Build clusters of similar titles
# Keep only pairs with Jaccard >= 0.4
jaccard_filtered = similar_df[similar_df['Sim'] >= 0.4].copy()

clusters = []
visited = set()

# We will treat each connected group of titles as a cluster
for t in pd.unique(jaccard_filtered[['Title1', 'Title2']].values.ravel()):
    if t in visited:
        continue

    # start a new cluster with this title
    group = {t}
    changed = True

    # keep adding titles that are linked to any title in the group
    while changed:
        changed = False
        for _, row in jaccard_filtered.iterrows():
            a, b = row['Title1'], row['Title2']
            if a in group and b not in group:
                group.add(b)
                changed = True
            elif b in group and a not in group:
                group.add(a)
                changed = True

    visited |= group
    clusters.append(group)

print("Number of clusters:", len(clusters))


Number of clusters: 9


In [31]:
# Choose one canonical title per cluster
def choose_canonical(group):
    return min(group, key=len)

mapping = {}

for group in clusters:
    canon = choose_canonical(group)
    for t in group:
        mapping[t] = canon

filtered_books['Title_final'] = filtered_books['Title_cleaned'].map(mapping).fillna(filtered_books['Title_cleaned'])


In [32]:
filtered_books = filtered_books[['Title_final', 'User_id', 'review/score']]
filtered_books

,Title_final,User_id,review/score
38660,gods and kings chronicles of the kings 1,A7IMBNFYANPNV,5.0
38661,gods and kings chronicles of the kings 1,A1WV4Q44JE40UF,5.0
38662,gods and kings chronicles of the kings 1,A32MYDPSMCHT1L,5.0
38663,gods and kings chronicles of the kings 1,AAOJ5T6VS5Z6O,5.0
38664,gods and kings chronicles of the kings 1,A2X1OYBRURS04R,5.0
...,...,...,...
2994351,first 100 words bright baby,AIFSOGUNXFADJ,5.0
2994352,first 100 words bright baby,A2B9B5F9FQ4JJ8,4.0
2994354,first 100 words bright baby,A2G9EZ3R716RH1,4.0
2994355,first 100 words bright baby,A37P45ZC5DSAJJ,5.0


###### *Create the Transition matrix M*

In [34]:
# For each pair of books, count how many users liked both
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix

# Remove duplicated (User_id, Title) pairs
df = filtered_books[["User_id", "Title_final"]].drop_duplicates()

# Convert User_id and Title into integer IDs
user_codes, user_index = pd.factorize(df["User_id"], sort=True)
title_codes, title_index = pd.factorize(df["Title_final"], sort=True)

n_users = len(user_index)
n_titles = len(title_index)

# Build matrix B (users x titles): B[u, t] = 1 if user u liked title t
data = np.ones(len(df), dtype=np.int8)
B = coo_matrix((data, (user_codes, title_codes)), shape=(n_users, n_titles)).tocsr()

# Shared-users matrix W (titles x titles): W[i, j] = #users who liked both titles
W = (B.T @ B).tocsr()

# Remove self-counts on the diagonal
W.setdiag(0)
W.eliminate_zeros()

print("W shape:", W.shape)
print("Nonzero edges:", W.nnz)

W shape: (358, 358)
Nonzero edges: 16834


In [35]:
# Keep only strong edges
W = W.multiply(W >= MIN_SHARED_USERS_FOR_EDGES)

W_df = pd.DataFrame(W.toarray(), index=title_index, columns=title_index)
W_df.head()

,13 little blue envelopes,1491 new revelations of the americas before columbus,1632 the assiti shards,1906,23 minutes in hell one mans story about what he saw heard and felt in that place of torment,500 lowcarb recipes 500 recipes from snacks to dessert that the whole family will love,a breath of snow and ashes outlander,a bride most begrudging,a caress of twilight meredith gentry book 2,a certain slant of light,...,who wrote the bible,why men love bitches from doormat to dreamgirl a womans guide to holding her own in a relationship,wicked the grimmerie a behindthescenes look at the hit broadway musical,wizards first rule sword of truth book 1,words that work its not what you say its what people hear,wuthering heights,yours until dawn,zanes afterburn a novel,zen in the martial arts,zen shorts caldecott honor book
13 little blue envelopes,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1491 new revelations of the americas before columbus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
1632 the assiti shards,0,0,0,0,0,0,0,0,2,0,...,0,0,0,3,0,0,0,0,0,0
1906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23 minutes in hell one mans story about what he saw heard and felt in that place of torment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Build transition probability matrix M
# W: sparse (n x n), W[i, j] = weight from book j to i
W = W.tocsc().astype(float)
n = W.shape[0]

col_sums = np.asarray(W.sum(axis=0)).ravel()
dangling = (col_sums == 0)

M = W.copy()

# Normalize non-dangling columns
for j in np.where(~dangling)[0]:
    start, end = M.indptr[j], M.indptr[j + 1]
    if start < end:
        M.data[start:end] /= col_sums[j]

# Fill dangling columns efficiently
if np.any(dangling):
    M = M.tolil()
    for j in np.where(dangling)[0]:
        M[:, j] = 1.0 / n
    M = M.tocsr()
else:
    M = M.tocsr()

col_check = np.asarray(M.sum(axis=0)).ravel()
print("M shape:", M.shape)
print("Column sums in [min, max]:", col_check.min(), col_check.max())

M shape: (358, 358)
Column sums in [min, max]: 0.9999999999999992 1.0000000000000064


##### **4. Apply PageRank-based Link Analysis**


---




*   Mathematical term: `v'= βMv + (1-β)e/n`



In [37]:
def pagerank(M, alpha=0.85, tol=1e-8, max_iter=100):
    if isinstance(M, pd.DataFrame):
        M = M.values

    N = M.shape[0]
    p = np.ones(N) / N
    teleport = np.ones(N) / N

    for _ in range(max_iter):
        p_new = alpha * (M @ p) + (1 - alpha) * teleport
        if np.linalg.norm(p_new - p, 1) < tol:
            break
        p = p_new

    return p / p.sum()

In [38]:
scores = pagerank(M)

# Attach book titles
pagerank_scores = pd.Series(scores, index=title_index, name='PageRank')

# Sort from most important book to least
pagerank_scores = pagerank_scores.sort_values(ascending=False)

pagerank_scores = pd.DataFrame(pagerank_scores)
pagerank_scores

,PageRank
exodus turtleback school library binding edition,0.042599
jane eyre large print,0.019620
jane eyre new windmill,0.019620
wuthering heights,0.017502
a tale of two cities literary touchstone edition,0.016087
...,...
the urantia book revealing the mysteries of god the universe jesus and ourselves,0.000498
the wealthy spirit daily affirmations for financial stress reduction,0.000498
algebra survival guide a conversational guide for the thoroughly befuddled,0.000498
adobe photoshop cs2 oneonone,0.000498
